In [1]:
import heapq

In [2]:
def add_task_difference(h, w, d): # duration, weight / priority
    heapq.heappush(h, (-(w-d), -w, d))

def add_task_proportion(h, w, d): # duration, weight / priority
    heapq.heappush(h, (-w/d, -w, d))

def sumup(h):
    d = 0
    s = 0
    while len(h) > 0:
        _, tw, td = heapq.heappop(h)
        d += td
        s += d * -tw
    print(s)

In [5]:
h1, h2 = list(), list()
with open('jobs.txt') as f:
    print("Reading {} records".format(f.readline()[:-1]))
    for line in f:
        w, d = (int(x) for x in line.split())
        add_task_difference(h1, w, d)
        add_task_proportion(h2, w, d)

Reading 10000 records


In [6]:
sumup(h1)
sumup(h2)

69119377652
67311454237


In [7]:
class Vertex:

    __slots__ = ('name', 'weight', 'incident_edges', 'heapidx')

    def __init__(self, name):
        self.name = name
        self.weight = (None, float('inf'))
        self.incident_edges = tuple()
        self.heapidx = None

    def connect(self, other, cost):
        self.incident_edges += ((other, cost),)

    def __lt__(self, other):
        return self.weight[1] < other.weight[1]

    def __le__(self, other):
        return self.weight[1] <= other.weight[1]

    def __gt__(self, other):
        return self.weight[1] > other.weight[1]

    def __ge__(self, other):
        return self.weight[1] >= other.weight[1]

    def __hash__(self):
        return hash(self.name)


class Graph:
    def __init__(self):
        self.V = dict()

    def heappush_vertex(self, h, v):
        j = len(h)
        i = j >> 1
        h.append(v)
        v.heapidx = j
        while j != 0 and h[j] < h[i]:
            h[j], h[i] = h[i], h[j]
            h[j].heapidx, h[i].heapidx = j, i
            j, i = i, i >> 1

    def heappop_vertex(self, heap, idx=0):
        if idx >= len(heap):
            raise IndexError('Not enough elements in heap')
        if idx == len(heap) - 1:
            return heap.pop()
        res, heap[idx] = heap[idx], heap.pop()
        heap[idx].heapidx = idx
        i, j = idx, idx + 1 if idx == 0 else idx << 1
        while j < len(heap):
            minj = j+1 if j+1 < len(heap) and heap[j] >= heap[j+1] else j
            if heap[i] <= heap[minj]:
                break
            heap[i], heap[minj] = heap[minj], heap[i]
            heap[minj].heapidx, heap[i].heapidx = minj, i
            i, j = minj, minj << 1
        return res

    def edge(self, u, v, c):
        if u not in self.V:
            self.V[u] = Vertex(u)
        if v not in self.V:
            self.V[v] = Vertex(v)
        self.V[u].connect(self.V[v], c)
        self.V[v].connect(self.V[u], c)

    def mst(self):
        vheap = list()
        x = set()
        for k in self.V:
            self.V[k].weight = (None, float('inf'))
            self.heappush_vertex(vheap, self.V[k])
        while len(x) != len(self.V):
            u = self.heappop_vertex(vheap)
            x.add(u)
            for (v, c) in u.incident_edges:
                if v not in x and c < v.weight[1]:
                    v.weight = (u, c)
                    self.heappush_vertex(vheap, self.heappop_vertex(vheap, v.heapidx))
        return {u.weight for u in x}

In [11]:
G = Graph()
with open('edges.txt') as f:
    print("{} V / E".format(f.readline()[:-1]))
    for line in f:
        u, v, c = line.split()
        G.edge(int(u), int(v), int(c))

500 2184 V / E


In [12]:
t = G.mst()

In [10]:
sum(e[1] for e in t if e[0] is not None)

-3612829

In [13]:
sum(e[1] for e in t if e[0] is not None)

-3612829